In [ ]:
import os
import csv
import argparse
import math
import pandas as pd
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg, NavigationToolbar2Tk)
import matplotlib.pyplot as plt
from tkinter import *

pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
# Input CSV filename from the command line
# parser = argparse.ArgumentParser()
# parser.add_argument("-d", "--directory", type=str, help="Path to data directory")
# parser.add_argument("-f", "--filename", type=str, help="CSV filename")
# parser.add_argument("-w", "--width", type=int, help="Window width")
# parser.add_argument("-l", "--height", type=int, help="Window height")
# args = parser.parse_args()

# Load the CSV as a DataFrame
# parent_path = os.path.dirname(os.path.abspath(__file__))
# csv_path = os.path.join(parent_path, args.directory, args.filename)

width = 500
height = 500
csv_path = "data/felt.csv"

print(f"Reading from: {csv_path}")
df = pd.read_csv(csv_path)

In [ ]:
# Sensor locations
#    r0_1	r2_1	r6_1	r0_2	r2_2	r6_2
x = [0,     -1,     4,      3,      1,      -2]
y = [0,     14,     13,     8,      16,     19]

In [ ]:
# Extract relevant columns
# r0_1 etc. are already the converted masses/weights
df_clean = df[['r0_1', 'r2_1', 'r6_1', 'r0_2', 'r2_2', 'r6_2']]
df_clean['x_com'] = 0
df_clean['y_com'] = 0
df_clean['weight_tot'] = 0

In [ ]:
# Not sure why my x_com and y_com are different than the csv
# But mine matches manual calculation
nrows, ncols = df_clean.shape
for i in range(nrows):
    weighted_x, weighted_y, weight_tot = 0, 0, 0
    for j in range(ncols-2):
        weight = df_clean.iloc[i, j]
        if not math.isinf(weight):
            weighted_x += weight * x[j]
            weighted_y += weight * y[j]
            weight_tot += weight
    df_clean.loc[i, 'x_com'] = weighted_x / max(weight_tot, 0.1)
    df_clean.loc[i, 'y_com'] = weighted_y / max(weight_tot, 0.1)
    df_clean.loc[i, 'weight_tot'] = weight_tot

In [ ]:
# Plot the FSR and COM locations
fsr_factor=1
com_factor=1/6

def plot():
    idx = frame.get()
    weights = df_clean.iloc[idx, :6]
    com = df_clean.iloc[idx, 6:]

    ax.clear()
    ax.set_xlim((-7, 24))   # -7, 9
    ax.set_ylim((-7, 24))   # -5, 24

    for i, weight in enumerate(weights):
        circle = plt.Circle((x[i], y[i]), weight*fsr_factor, color='black')
        ax.add_patch(circle)
    circle = plt.Circle((com[0], com[1]), com[2]*com_factor, color='g')
    ax.add_patch(circle)

    canvas.draw()
    canvas.get_tk_widget().pack()
    toolbar.update()
    canvas.get_tk_widget().pack()

In [ ]:
window = Tk()
window.title("Foot Pressure")
window.geometry(f'{width}x{height}')

fig, ax = plt.subplots()
canvas = FigureCanvasTkAgg(fig, master=window)
toolbar = NavigationToolbar2Tk(canvas, window)

frame = Scale(window, from_=0, to=nrows-1, orient=HORIZONTAL)
frame.pack()

update_button = Button(master=window, command=plot, height=2, width=10, text="Update")
update_button.pack()

window.mainloop()